# Sentinel-1 EW RTC with pyroSAR (GAMMA)

This notebook is designed to be used on the National Computational Infrastructure (NCI) only, due to its use of the licensed GAMMA software.

The NCI project folder has the required symlinks for the GAMMA binaries. 

PyroSAR's GAMMA Python API must be built before running. Please run [initialise_gamma.py](initialise_gamma.py) after loading the conda environment for this project. This needs to be done only once.

**Requirements**
- A conda environment setup as described in the *README.md*
- Appropriate storage space
- Access to dg9 project on NCI
- Credentials for usgs (https://urs.earthdata.nasa.gov/users/new) 
- Credentials for copernicus CDSE ( https://dataspace.copernicus.eu/)

## Imports

In [ ]:
import logging
import sys

logging.basicConfig(
    format="%(asctime)s | %(levelname)s : %(message)s",
    level=logging.INFO,
    stream=sys.stdout,
)

log = logging.getLogger("gammapy")
log.setLevel(logging.INFO)

In [ ]:
import asf_search as asf
import os
import yaml
from pyroSAR import identify
from pyroSAR.gamma import geocode
from pyroSAR.gamma.api import diff
from pathlib import Path

import logging
import sys

## Settings

### Paths

In [ ]:
REPO_DIR = Path("/g/data/yp75/projects/pyrosar_processing/s1-rtc-pyrosar-notebook").resolve()
PROJ_DIR = REPO_DIR.parent
DATA_DIR = PROJ_DIR.joinpath("data")
CRED_DIR = REPO_DIR.joinpath("credentials")
SCENE_DIR = DATA_DIR.joinpath("scenes/ASF")
RESULTS_DIR = DATA_DIR.joinpath("results")
DEM_DIR = DATA_DIR.joinpath("dem")
TEMP_DIR = SCENE_DIR.joinpath("unzipped_scenes")
LOG_DIR = DATA_DIR.joinpath("logs")
ORBIT_DIR = DATA_DIR.joinpath("orbits")

#### Create paths if not exist

In [ ]:
directories_list = [val for key,val in locals().items() if "DIR" in key]

create_dirs = True
if create_dirs:
    for directory in directories_list:
        print(directory)
        directory.parent.mkdir(parents=True, exist_ok=True)


### GAMMA Environment Variables

In [ ]:
GAMMA_HOME_PATH = "/g/data/dg9/GAMMA/GAMMA_SOFTWARE-20230712"

gamma_env_value = os.environ.get("GAMMA_HOME", None)

if gamma_env_value is None:
        os.environ["GAMMA_HOME"] = GAMMA_HOME_PATH
        gamma_env_value = os.environ["GAMMA_HOME"]

### Shared Objects Environment Variable
Important to check the current directory for symlinks to gdal and fftw3, which are required to run the GAMMA binaries

In [ ]:
REQUIRED_LIBS_PATH = "/g/data/yp75/projects/pyrosar_processing/s1-rtc-pyrosar-notebook:/apps/fftw3/3.3.10/lib:/apps/gdal/3.6.4/lib64"

ld_lib_env_value = os.environ.get("LD_LIBRARY_PATH", None)

if ld_lib_env_value is None:
        os.environ["LD_LIBRARY_PATH"] = REQUIRED_LIBS_PATH
        ld_lib_env_value = os.environ["LD_LIBRARY_PATH"]

### Credentials information

In [ ]:
EARTHDATA_CREDS_PATH = CRED_DIR.joinpath("credentials_earthdata.yaml")

### Scene information

In [ ]:
SCENE_ID = "S1A_EW_GRDM_1SDH_20220117T122010_20220117T122115_041500_04EF6B_6437"
MODE = "EW"
PRODUCT = "GRD_MD"

scene_zip = SCENE_DIR.joinpath(f"{SCENE_ID}.zip")

### Existing DEM and DEM settings

In [ ]:
dem = DEM_DIR.joinpath(f"{SCENE_ID}_dem") # Binary format, used by GAMMA
dem_src = DEM_DIR.joinpath(f"{SCENE_ID}_dem.tif") # Source file, GeoTIFF
dem_gamma = DEM_DIR.joinpath(f"{SCENE_ID}_dem.par") # GAMMA Parameter file

## Processing

In [ ]:
# get scene metadata
if scene_zip.exists():
    pyrosar_scene_id = identify(scene_zip)
else:
    print("Scene not available, download first")

### DEM creation
GAMMA requires specific file formats, and pyroSAR currently provides this
through dem_autocreate, which does involve doing download of DEM files

In [ ]:
if dem_gamma.exists():
    log.info("DEM exists")
    pass
else:
    geoid_str = 'EGM2008' # suggested in dem_autocreate for copernicus 30m dem
    geoid_path = Path(GAMMA_HOME_PATH).joinpath('DIFF/scripts/egm2008-5.dem')
    # Function
    log.info("running DEM")
    diff.dem_import(
        input_DEM=str(dem_src), 
        DEM=str(dem),
        DEM_par=str(dem_gamma),
        no_data=-9999,
        geoid=geoid_path, 
        logpath=str(LOG_DIR), 
        outdir=str(DEM_DIR)
    )
    log.info("finished DEM")

### GAMMA Geocode

In [ ]:
# Settings
spacing = 40
scaling = 'linear' # scale of final product, linear, db
refarea = 'gamma0' # e.g. gamma0, sigma0, beta0 or ['gamma0','sigma0']

In [ ]:
log.info("running geocode")
geocode(
    scene=pyrosar_scene_id, 
    dem=str(dem), 
    tmpdir=str(TEMP_DIR),
    outdir=str(RESULTS_DIR), 
    spacing=spacing, 
    scaling=scaling, 
    func_geoback=1,
    nodata=(0, -99), 
    update_osv=True, 
    osvdir=ORBIT_DIR, 
    allow_RES_OSV=True,
    cleanup=False, 
    export_extra=None, 
    basename_extensions=None,
    removeS1BorderNoiseMethod='gamma', 
    refine_lut=False, 
    rlks=None, 
    azlks=None,
    s1_osv_url_option=1
)
log.info("finished geocode")